In [2]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,0,28,2024-07-01 20:02:19
1,大安區,21,11,10,2024-07-01 19:58:17
2,大安區,16,7,9,2024-07-01 19:45:21
3,大安區,11,1,10,2024-07-01 19:59:20
4,大安區,16,3,13,2024-07-01 20:01:22
...,...,...,...,...,...
1424,臺大公館校區,30,0,29,2024-07-01 20:01:22
1425,臺大公館校區,20,2,18,2024-07-01 20:02:19
1426,臺大公館校區,24,3,20,2024-07-01 20:02:19
1427,臺大公館校區,40,28,3,2024-07-01 20:02:19


In [4]:
df1 = df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg([('車輛數','sum'),('站點數','count')])
df1

總數量       可借數量       可還數量     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
行政區域                                   
中山區     3819  156  1535  156  2254  156
中正區     3209  121  1517  121  1665  121
信義區     3331  106  1293  106  1934  106
內湖區     3499  168  1451  168  2009  168
北投區     2472   94   916   94  1526   94
南港區     2317   92  1096   92  1198   92
士林區     2937  127  1079  127  1784  127
大同區     1629   62   569   62  1046   62
大安區     4987  185  2139  185  2751  185
文山區     2203   97   824   97  1376   97
松山區     2343   88   921   88  1404   88
臺大公館校區  1437   53   530   53   872   53
萬華區     2057   80  1007   80  1025   80

In [8]:

#pivot
#stack,unstack
#欄位有名稱
s1 = df1.stack(level=[0,1])
s1.index.names = ['行政區域','主題','數量']
s1.unstack(level=['數量'])

/tmp/ipykernel_13691/3376254337.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  s1 = df1.stack(level=[0,1])


數量            車輛數  站點數
行政區域   主題             
中山區    可借數量  1535  156
       可還數量  2254  156
       總數量   3819  156
中正區    可借數量  1517  121
       可還數量  1665  121
       總數量   3209  121
信義區    可借數量  1293  106
       可還數量  1934  106
       總數量   3331  106
內湖區    可借數量  1451  168
       可還數量  2009  168
       總數量   3499  168
北投區    可借數量   916   94
       可還數量  1526   94
       總數量   2472   94
南港區    可借數量  1096   92
       可還數量  1198   92
       總數量   2317   92
士林區    可借數量  1079  127
       可還數量  1784  127
       總數量   2937  127
大同區    可借數量   569   62
       可還數量  1046   62
       總數量   1629   62
大安區    可借數量  2139  185
       可還數量  2751  185
       總數量   4987  185
文山區    可借數量   824   97
       可還數量  1376   97
       總數量   2203   97
松山區    可借數量   921   88
       可還數量  1404   88
       總數量   2343   88
臺大公館校區 可借數量   530   53
       可還數量   872   53
       總數量   1437   53
萬華區    可借數量  1007   80
       可還數量  1025   80
       總數量   2057   80